In [1]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from config import Config
from model_vc import Generator

#Temporary:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #Use gpu or cpu

def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

device = torch.device('cuda')
G = Generator(32,256,512,32).eval().to(device)

g_checkpoint = torch.load('../networks/autovc.ckpt', map_location=device)
G.load_state_dict(g_checkpoint['model'])

metadata = pickle.load(open('metadata.pkl', "rb"))

spect_vc = []

for sbmt_i in metadata:
    print(sbmt_i)          
    x_org = sbmt_i[2]
    x_org, len_pad = pad_seq(x_org)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata:
                   
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        


with open('results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)          

['p225', array([-2.92885415e-02,  1.67739280e-02,  6.42375797e-02,  5.12384847e-02,
        8.77934247e-02, -4.05867286e-02,  1.08849108e-02,  2.06673276e-02,
       -8.39690343e-02,  1.48199843e-02,  3.82344425e-02,  1.12243919e-02,
        4.62971367e-02,  1.66661311e-02, -5.78785129e-02,  3.60288732e-02,
        1.92339886e-02,  9.17971320e-03,  2.74707917e-02, -5.48039749e-02,
        2.50798557e-02,  4.66737375e-02, -6.14981353e-03,  3.88026945e-02,
        5.68139665e-02, -7.33052716e-02,  2.30920967e-02, -1.04292825e-01,
       -2.61898227e-02,  3.02257240e-02, -3.02889403e-02,  3.63447554e-02,
       -4.97230627e-02,  1.57715172e-01, -3.93295921e-02,  5.51161245e-02,
        4.64604087e-02, -4.59927395e-02, -4.82378080e-02, -3.81431282e-02,
        3.91379185e-02,  4.59317304e-02, -1.55072343e-02, -1.67513415e-02,
       -5.09507731e-02, -5.08496165e-02, -2.25679334e-02,  2.63878461e-02,
       -1.26611767e-02, -2.49883570e-02,  3.02621610e-02, -2.48487398e-04,
       -1.226522